## Converting from Codepoint to Char and Back

In [ ]:
# wrong way
[char]103
[char]0x1f468

g
InvalidArgument: 
Line |
   3 |  [char]0x1f468
     |  ~~~~~~~~~~~~~
     | Cannot convert value "128104" to type "System.Char". Error: "Value was either too large or too small for a character."


In [ ]:
'was it too big to fit into one [char]?'
0x1f468 -gt [char]::MaxValue

was it too big to fit into one [char]?
True


In [ ]:
# Codepoint (int32) to char/string
[char]::ConvertFromUtf32( 0x1f468 )

# codepoint from [char] to codepoint
$charList = '👨'.ToCharArray()
[char]::ConvertToUtf32( $charList[0], $charList[1] )

# it outputs in decimal, it's the same number
128104 -eq 0x1f468


👨
128104
True


Note: The return type of `[char]::ConvertFromUtf32` is actually a `[string]`, not a `[char]`

In [ ]:
function inspectString {
    param(
        [string]$InputText
    )
    
    $info = @{
        'Input' = $InputText
        'String.Length' = $InputText.Length        
        # 'Utf8SequenceLength' = $stats.Utf8SequenceLength.Sum
        # 'Utf16SequenceLength' = $stats.Utf16SequenceLength.Sum
        # 'Utf8SequenceCount' = $stats.Utf8SequenceLength.count
        # 'Utf16SequenceCount' = $stats.Utf16SequenceLength.Count
        'TotalCodepoints' = $inputText.EnumerateRunes().Value.count
        'Details' = $InputText.EnumerateRunes()
    }
    # | Ft -AutoSize Property, Sum, Count

    return [pscustomobject]$Info
    # return $Info
}

$str = '👨👨‍👩‍👧‍👦'
inspectString $str | ft -auto
$strMan = '👨'
inspectString $strMan | ft -auto

$str.EnumerateRunes()
    | measure -Property Utf8SequenceLength, Utf16SequenceLength -Sum
    | ft Property, Sum, Count


Input         Details          String.Length TotalCodepoints
-----         -------          ------------- ---------------
👨👨‍👩‍👧‍👦 {👨, 👨, ‍, 👩…}            13               8


Input Details String.Length TotalCodepoints
----- ------- ------------- ---------------
👨    {👨}                2               1


Property            Sum Count
--------            --- -----
Utf8SequenceLength   29     8
Utf16SequenceLength  13     8



In [ ]:
(inspectString $str ).Details | Ft -auto


IsAscii IsBmp Plane Utf16SequenceLength Utf8SequenceLength  Value
------- ----- ----- ------------------- ------------------  -----
  False False     1                   2                  4 128104
  False False     1                   2                  4 128104
  False  True     0                   1                  3   8205
  False False     1                   2                  4 128105
  False  True     0                   1                  3   8205
  False False     1                   2                  4 128103
  False  True     0                   1                  3   8205
  False False     1                   2                  4 128102



In [ ]:
# 'other numbers'
$str.Length
[Text.Encoding]::UTF8.GetByteCount( $str ) # Byte of char

# [Text.Encoding]::Unicode.GetByteCount( $bstr )        
$bStr_16 = [Text.Encoding]::Unicode.GetBytes( $str ) # char of bytes
$bStr_16.Length
[Text.Encoding]::Unicode.GetCharCount( $bstr_16 ) #char of bytes

13
29
26
13


Converting a codepoint to a `[char]`, (actually a `[string]`)


Assume that Dotnet uses the encoding `UTF-16LE` by default, unless the docs explicitly say another. 

| Type | Description |
| - | -  |
| `[char]` | represesnts a single UTF16le `code-unit`<br>max value is **2-bytes** |
| `[String]` | a list of utf16le code-units | 
| `[Rune]` | represents a single Unicode Scalar Codepoint<br>in the range `[0..0x10ffff]`<br>Except the Surrogate range `[0xd800..0xdfff]`   |

docs:

- [Introduction to Character Encoding in `.NET`](https://docs.microsoft.com/en-us/dotnet/standard/base-types/character-encoding-introduction)
- [`[Char]`](https://docs.microsoft.com/en-us/dotnet/api/system.char?view=net-6.0)
- [`[String]`](https://docs.microsoft.com/en-us/dotnet/api/system.char?view=net-6.0)
- [`[Text.Rune]`](https://docs.microsoft.com/en-us/dotnet/api/system.text.rune?view=net-6.0)
- [`[Globalization.StringInfo]`](https://docs.microsoft.com/en-us/dotnet/api/system.globalization.stringinfo?view=net-6.0)

Note: **WindowsPowershell** does not have any `[Rune]` functions

See more:
- [Introduction to Character Encoding in `.NET`](https://docs.microsoft.com/en-us/dotnet/standard/base-types/character-encoding-introduction)
- [`System.String` and `char`](https://docs.microsoft.com/en-us/dotnet/api/system.string?view=net-6.0#char-objects-and-unicode-characters)